In [1]:
import numpy as np
import awkward
import uproot_methods

In [53]:
nevents = 50000
counts = np.minimum(np.random.exponential(2, size=nevents).astype(int), 20)

nobj = np.sum(counts)
p4 = np.empty((nobj, 4))
p4[:,:2] = np.random.normal(0., 20., size=nobj*2).reshape((-1, 2))
p4[:,2] = np.sqrt(p4[:,0]**2+p4[:,1]**2)*np.sinh(np.random.uniform(-2.5, 2.5, size=nobj))
p4[:,3] = np.sqrt(np.sum(p4[:,:3]**2, axis=1)+0.000511)

q = np.random.randint(2, size=nobj)*2-1
mva = np.random.normal(size=nobj)

In [105]:
JaggedWithLorentz = awkward.Methods.mixin(uproot_methods.classes.TLorentzVector.ArrayMethods, awkward.JaggedArray)
#ObjectArrayWithLorentz = awkward.Methods.mixin(uproot_methods.classes.TLorentzVector.ArrayMethods, awkward.ObjectArray)
form = lambda input_dict: awkward.JaggedArray.fromcounts(counts, awkward.Table(input_dict))

objects = form({
        'p4': uproot_methods.TLorentzVectorArray(*tuple(p4[:,i] for i in range(4))),
        'q': q,
        'id': mva,
    })

In [106]:
good = objects['id'] > 0.1

In [111]:
pairs = objects.argpairs()
objects[pairs['0']]

IndexError: jagged array used as index contains out-of-bounds values

In [112]:
objects

<JaggedArray [[<Row 0> <Row 1>] [<Row 2> <Row 3>] [] ... [<Row 77318> <Row 77319>] [] []] at 7f2f6c082a50>

In [173]:
nevents = 50000
counts = np.minimum(np.random.exponential(2, size=nevents).astype(int), 20)
a = awkward.JaggedArray.fromcounts(counts, np.random.normal(size=np.sum(counts)))
b = awkward.JaggedArray.fromcounts(counts, np.random.normal(size=np.sum(counts)))
pairs = a.argpairs()
good = pairs.empty_like()
good.content = b.content[pairs['0'].flatten()]*2 > b.content[pairs['1'].flatten()]
result = good[good].empty_like()
result.content = a.content[pairs[good]['0'].flatten()] + a.content[pairs[good]['1'].flatten()]

In [174]:
pairs2 = a.pairs()
good2 = b.pairs()['0']*2 > b.pairs()['1']
result2 = pairs2[good2]['0'] + pairs2[good2]['1']

In [175]:
result2[:3]

<JaggedArray [[-1.55653048] [] [0.25908337 0.63805893 1.01703448]] at 7f2f6c2fa890>

In [176]:
result[:3]

<JaggedArray [[-1.55653048] [] [0.25908337 0.63805893 1.01703448]] at 7f2f6c2fa690>

In [181]:
np.all((result == result2).flatten())

True